Merging is the process of copying changes from one branch to another. An important thing to note here is that this is only true concept-wise. The actual implementation is efficient. If there's a commit that can be re-used, obviously git is just going to re-use that with multiple pointers instead of actually copying changes. As far as we are concerned, we're just talking about the concept.

There are two ways git does merging.

a fast forward merge is used, when the target branch is a complete subset of the source branch.

(shows picture)

in this example, since new-branch contains all the nodes that the master branch currently has, it is capable of performing a fast forward merge, which is fairly straightforward. The commits just get copied over to the master branch.

(shows picture)

otherwise, a three way merging is used.

take a look at this example. While you're developing on your branch, somebody else has updated changes into the master branch. So in this case, the branches have diverged. What git will do when you merge new-branch into master, is that git will create a merged commit on the target branch, and that merged node will contain all the changes that you've done on new-branch in one single commit.

Now let's see these in action.

First, a fast forward merge. We're gonna checkout the target branch, and then we're gonna merge content of new-branch into my current checked out branch.

git checkout master
git merge new-branch

As you can see, a fastforward merge is done. git log should show that both master and new-branch are now pointing to the same node. master is now updated.

git log

now, let's recreate the situation where master has diverged from new-branch. now, we're gonna do a bit of hacking here, i'm not explain the steps that I'm about to do, because it is actually out of context as to what I wanted to show you.

first copy down the SHA of old master
git reset SHA_master --hard

okay now we're back. Now, I'm gonna checkout master branch, and start adding something different.

(open one of the files, and add a non-conflicting line)
git commit -m "first commit after branching off on master"

now, let's try the merge:

git checkout master
git merge new-branch

(shows screen)

and now because the master and new-branch have diverged, a fast-forward merge can no longer be used. A three way merge is being used here, as indicated by the recursive strategy. And as you can see, the files now contain both changes from master as well as new-branch.

So what is a three way merge? Why is it called a three way merge, when we're only merging two versions of the same file. We've got version A from master branch, and version B from new-branch.

(shows picture)

Take a look at this code. Code on the left, let's say that's on the master branch. Code on the right, let's say that's on the new-branch. Now, just by looking at these two files alone, I can't tell what changes have been made by the user.

(shows picture)

Let's say the user started off with the above. Then the one of the left made these changes, the one on the right made these ones. That's one possibility.

(shows picture)

Here's another possibility. one on the left added these changes, one of the right added these lines.

(shows picture)

Here's another possibility, one on the left deleted these lines, one on the right deleted some other lines.

I don't know exactly what changes have been made. You can't be sure either. Git certainly does not know either, if it only had these two files to compare with.

So hence why a three way merge is being used. A three way merge needs three files, a base file, changed version of the file by the first branch, and the changed version of the file by the second branch. The base is the last common commit, or the last common version of the file between the two branches. What git actually does in the three way merge, is that it compares the first changed version to the base file, then compares the second changed version to the base file. So now it knows exactly what has been changed by each branch, respectively.

This concept is important, the base, version A and version B used in a three way merge by git. Because now we're going to do some resolve some merge conflicts. We're not gonna use any external diff tools, and we're gonna do this the manual way. Just by editing the conflicting text file from a merge.

conflict resolution:

So it's not always a happy world. It's quite possible that somebody has changed the exact same file which you've been working on at the exact same place, but with different content. So we need to know how git resolves conflicts. It's quite simple once you've understood three way merge mechanism.

First, let's create a situation where conflict happens. and to do that, I need to perform some hacking and undo the merge commit created from earlier. Ignore this, this is not related to what I'm showing you here.

git reset --hard SHA_of_commit_before_merge_on_master
git checkout master

now let's edit the files so they have conflicting changes.

(edits the file)

git commit -m "this is the commit that is going to cause conflict"

(shows version of the files on master branch)
(shows version of the files on new-branch)

now we're going to cause some problems.

git checkout master
git merge new-branch

BOOM. read the conflicting message. Automatic merge has failed, fix the files, then commit the result. We're gonna take a look at the conflicting files and see what git has generated for us.

(opens file)

so this is what git puts in your file when it detects a conflict. I'm gonna have to explain what a HEAD is. It's really quick, HEAD is whatever branch you're on right now, basically. You have many different branches, but which branch are you currently looking at? That's where the HEAD pointer is. So when you do git checkout master, git puts the HEAD pointer on master. This basically means master. you can substitute the word master here.

let's say you want first version, delete all this crap here. resolved.
let's say you want second version.
let's say you want both first and second version.
let's say you don't want any of this.
let's say you see these two versions, and be like, this is not what I want, I want something else, I want something custom.

on to the next file
puts (master) beside HEAD
master says blah
new-branch says blah2

let's say i want the line to actually make sense. delete all the marking.

now, we're done all the conflict resolution.
head back into the console, and here I wanna show you something interesting here.
If you think you screwed up, and you didnt mean to merge. What this process here right now, is actually a git merge in progress. If you wanna revert to what it was like before the merge, simply do

git merge --abort

and we're back to the peaceful world again. we're back to where things were before, to the state before the dreadful merge. you don't need to panic.

now we're gonna head back into the chaos by doing:

git checkout master
git merge new-branch

let's do the same thing.
(edits the file, saves)

okay, now let's proceed with the merge.
git commit

committing is not possible, because you have conflicts. Oh, I forgot to stage my changes.
git add *
git commit

remember, no message is needed here. An automatic merging message will be generated for you.

git log

and these are the final versions of the file, which we have already been reviewed by us. So we know exactly what the outcome is for the merge.

closing words:

that's all the content I want to show you today. There will be a follow up session next week or so. I will be going more in depth into git, particularly heavy on git rebase.

I'm gonna show you another way of doing merging, which is merging by git rebase versus the traditional git merge. The end result is the same, as far as workcopy and files that is stored in the version control system, those are the same. But the way that git does the merge is completely different between rebase and merge. I'm gonna show you the pros and cons of each, and why do developers prefer rebase.

Then I'm gonna show you some past examples of my own contributions into a bunch of repositories on github, and this includes repos from wordpress, mozilla and NASA. I'm gonna briefly show you how you can get started with open source development, taking on an issue, forking, investigate, change code, how to create a pull request, request reviews, interact with developers, modify code upon requests, and after they're satisfied, squash your commits by rebasing before getting it merged.

git rebase is an advanced command. This does not mean that you're a better developer. Advanced command means you're modifying existing commit history of records. And if you screw up, and you do not have a good back up somewhere, you're gonna have to recreate the work that you've doing manually from scratch. So it is an advanced command because it is dangerous.

But it is an incredibly powerful command, you can delete commits from history, you can squash multiple commits into one, you can edit the message of an old commit. I think you can even edit the content of an old commit. Imagine somebody committed a bunch of files. You go in, and do git rebase, which affects this commit. You accidentally delete one of the files from his commit. This is permanent. This is not recoverable. This is not something where oh, I'll just do a git checkout to a previous commit so I can recover the file. No, you changed the history, you changed the record, and the file is not there anymore. hence why git rebase needs to be used with care.
